Run `pip install .` first to install all dependencies.

In [1]:
!pip install gcsfs==2022.10.0
!pip install google-auth==2.15.0
!pip install psutil==5.9.4
!pip install wandb
!pip install fastparquet
!pip install numpy
!pip install pandas
!pip install catboost
!pip install scipy

Defaulting to user installation because normal site-packages is not writeable
  Using cached gcsfs-2022.10.0-py2.py3-none-any.whl (25 kB)
  Using cached fsspec-2022.10.0-py3-none-any.whl (138 kB)
  Using cached google_cloud_storage-2.7.0-py2.py3-none-any.whl (110 kB)
  Using cached google_auth_oauthlib-0.8.0-py2.py3-none-any.whl (19 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Using cached google_resumable_media-2.4.0-py2.py3-none-any.whl (77 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
  Using cached protobuf-4.21.12-cp37-abi3-manylinux2014_x86_64.whl (409 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.0/218.0 KB 10.5 MB/s eta 0:00:00
  Using cached google_crc32c-1.5.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32 kB)
ERROR: Will not install to the user site because it will lack sys.path precedence to fsspec in /pfs/data5/software_uc2/bwhpc/common

In [2]:
import os

from catboost import CatBoostClassifier, Pool
import numpy as np

import gcsfs
import numpy as np
import pandas as pd

import wandb

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, PowerTransformer
from sklearn.exceptions import NotFittedError

from scipy import stats

In [3]:
# # connect to weights and biases
# run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")


In [4]:
# import google.auth
# from google.colab import auth
# # connect to google cloud storage
# auth.authenticate_user()
# credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis")
#fs = gcsfs.GCSFileSystem(project="thesis", credentials=credentials)

/home/kit/stud/uloak/.local/lib/python3.8/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
# reduce number of imported cols due to memory issues
columns = [
    "QUOTE_DATETIME",
    "ROOT",
    "EXPIRATION",
    "STRK_PRC",
    "OPTION_TYPE",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_BID",
    "BEST_ASK",
    "ask_ex",
    "bid_ex",
    "bid_size_ex",
    "ask_size_ex",
    "price_all_lead",
    "price_all_lag",
    "price_ex_lead",
    "price_ex_lag",
    "buy_sell",
]


In [31]:
train = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_extended_60.parquet",
    engine="fastparquet",
    columns=columns,
)
val = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/val_set_extended_20.parquet",
    engine="fastparquet",
    columns=columns,
)

In [34]:
num_features = [
    "STRK_PRC",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_BID",
    "BEST_ASK",
    "ask_ex",
    "bid_ex",
    "bid_size_ex",
    "ask_size_ex",
    "price_all_lead",
    "price_all_lag",
    "price_ex_lead",
    "price_ex_lag",
    "day_vol", 
    "myn"
]

## Yeo Johnson Test

In [35]:
yeo_johnson = PowerTransformer(method="yeo-johnson")
yeo_johnson.fit(train[num_features])

PowerTransformer()

In [36]:
lambdas = pd.Series(data=yeo_johnson.lambdas_, index=num_features)
lambdas

STRK_PRC         -0.126414
TRADE_SIZE       -0.379786
TRADE_PRICE      -0.446928
BEST_BID         -0.442183
BEST_ASK         -0.442264
ask_ex           -0.441659
bid_ex           -0.441708
bid_size_ex       0.035735
ask_size_ex      -0.011599
price_all_lead   -0.447201
price_all_lag    -0.451723
price_ex_lead    -0.449795
price_ex_lag     -0.453784
dtype: float64

## Box Cox Test

In [37]:
train[num_features].min()

STRK_PRC          0.50
TRADE_SIZE        1.00
TRADE_PRICE       0.01
BEST_BID          0.00
BEST_ASK          0.00
ask_ex            0.00
bid_ex            0.00
bid_size_ex       0.00
ask_size_ex       0.00
price_all_lead    0.01
price_all_lag     0.01
price_ex_lead     0.01
price_ex_lag      0.01
dtype: float64

In [38]:
box_cox = PowerTransformer(method="box-cox")
# add constant as box cox works only on positive data
box_cox.fit(train[num_features]+1)

PowerTransformer(method='box-cox')

In [39]:
lambdas = pd.Series(data=box_cox.lambdas_, index=num_features)
lambdas

STRK_PRC         -0.126414
TRADE_SIZE       -0.379786
TRADE_PRICE      -0.446928
BEST_BID         -0.442183
BEST_ASK         -0.442264
ask_ex           -0.441659
bid_ex           -0.441708
bid_size_ex       0.035735
ask_size_ex      -0.011599
price_all_lead   -0.447201
price_all_lag    -0.451723
price_ex_lead    -0.449795
price_ex_lag     -0.453784
dtype: float64

In [42]:
box_cox = PowerTransformer(method="box-cox")
# add constant as box cox works only on positive data
box_cox.fit(train[num_features]+0.01)

PowerTransformer(method='box-cox')

In [43]:
lambdas = pd.Series(data=box_cox.lambdas_, index=num_features)
lambdas

STRK_PRC         -0.082968
TRADE_SIZE       -0.204893
TRADE_PRICE       0.060425
BEST_BID          0.110299
BEST_ASK          0.056837
ask_ex            0.049090
bid_ex            0.113067
bid_size_ex       0.140284
ask_size_ex       0.029876
price_all_lead    0.054975
price_all_lag     0.049097
price_ex_lead     0.053774
price_ex_lag      0.042910
dtype: float64

## Prepare dataset

In [ ]:
def sin_encode(x, period):
    return np.sin(x * 2 * np.pi / period)

def cos_encode(x, period):
    return  np.cos(x * 2 * np.pi / period)


In [21]:
scaler = StandardScaler()
oe_option_type = OrdinalEncoder(
    unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
)
oe_root = OrdinalEncoder(
    unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
)
oe_issue_type = OrdinalEncoder(
    unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
)


def transform(data: pd.DataFrame) -> pd.DataFrame:

    # set up df, overwrite later
    x = pd.DataFrame(data={"TRADE_PRICE": data["TRADE_PRICE"]}, index=data.index)

    # size features
    x["bid_ask_size_ratio_ex"] = data["bid_size_ex"] / data["ask_size_ex"]
    x["rel_bid_size_ex"] = data["TRADE_SIZE"] / data["bid_size_ex"]
    x["rel_ask_size_ex"] = data["TRADE_SIZE"] / data["ask_size_ex"]
    x["depth_ex"] = data["bid_size_ex"] - data["ask_size_ex"]

    # classical
    mid_ex = 0.5 * (data["ask_ex"] + data["bid_ex"])
    mid_best = 0.5 * (data["BEST_ASK"] + data["BEST_BID"])
    spread_ex = data["ask_ex"] - data["bid_ex"]
    spread_best = data["BEST_ASK"] - data["BEST_BID"]

    x["prox_ex"] = (data["TRADE_PRICE"] - mid_ex) / (0.5 * spread_ex)
    x["prox_best"] = (data["TRADE_PRICE"] - mid_best) / (0.5 * spread_best)

    # custom features
    x["spread_ex"] = spread_ex
    x["spread_best"] = spread_best
    x["bid_ask_ratio_ex"] = data["bid_ex"] / data["ask_ex"]
    x["price_rel_nbo"] = (data["TRADE_PRICE"] - data["BEST_ASK"]) / (
        data["BEST_ASK"] - mid_best
    )
    x["price_rel_nbb"] = (data["TRADE_PRICE"] - data["BEST_BID"]) / (
        mid_best - data["BEST_BID"]
    )

    # calculate change
    x["chg_ex_lead"] = data["TRADE_PRICE"] - data["price_ex_lead"]
    x["chg_ex_lag"] = data["TRADE_PRICE"] - data["price_ex_lag"]
    x["chg_all_lead"] = data["TRADE_PRICE"] - data["price_all_lead"]
    x["chg_all_lag"] = data["TRADE_PRICE"] - data["price_all_lag"]

    # log transformed features
    x[
        [
            "ask_ex",
            "bid_ex",
            "BEST_ASK",
            "BEST_BID",
            "TRADE_PRICE",
            "price_all_lag",
            "price_all_lead",
            "price_ex_lag",
            "price_ex_lead",
            "TRADE_SIZE",
            "bid_size_ex",
            "ask_size_ex",
            "day_vol",
            "myn",
            "STRK_PRC",
        ]
    ] = np.log1p(
        data[
            [
                "ask_ex",
                "bid_ex",
                "BEST_ASK",
                "BEST_BID",
                "TRADE_PRICE",
                "price_all_lag",
                "price_all_lead",
                "price_ex_lag",
                "price_ex_lead",
                "TRADE_SIZE",
                "bid_size_ex",
                "ask_size_ex",
                "day_vol",
                "myn",
                "STRK_PRC",
            ]
        ]
    )

    x["ttm"] = (
        data["EXPIRATION"].dt.to_period("M") - data["QUOTE_DATETIME"].dt.to_period("M")
    ).apply(lambda x: x.n)

    # save num columns for scaler
    num_cols = x.columns.tolist()

    # date features
    x["date_year"] = data["QUOTE_DATETIME"].dt.year

    months_in_year = 12
    x["date_month_sin"] = sin_encode(data["QUOTE_DATETIME"].dt.month, months_in_year)
    x["date_month_cos"] = cos_encode(data["QUOTE_DATETIME"].dt.month, months_in_year)

    days_in_week = 7
    x["date_weekday_sin"] = sin_encode(
        data["QUOTE_DATETIME"].dt.dayofweek, days_in_week
    )
    x["date_weekday_cos"] = cos_encode(
        data["QUOTE_DATETIME"].dt.dayofweek, days_in_week
    )

    seconds_in_day = 24 * 60 * 60
    seconds = (
        data["QUOTE_DATETIME"] - data["QUOTE_DATETIME"].dt.normalize()
    ).dt.total_seconds()

    x["date_time_sin"] = sin_encode(seconds, seconds_in_day)
    x["date_time_cos"] = cos_encode(seconds, seconds_in_day)

    # impute with zeros
    x.replace([np.inf, -np.inf], np.nan, inplace=True)
    x.fillna(0, inplace=True)

    # standardize continous columns (w/o date features)
    # bin encode categorical features
    try:
        x[num_cols] = scaler.transform(x)
        x["bin_option_type"] = oe_option_type.transform(
            data["OPTION_TYPE"].astype(str).values.reshape(-1, 1)
        )
        x["bin_issue_type"] = oe_issue_type.transform(
            data["issue_type"].astype(str).values.reshape(-1, 1)
        )
        x["bin_root"] = oe_root.transform(
            data["ROOT"].astype(str).values.reshape(-1, 1)
        )
        print("transform (val + test)")
    except NotFittedError as e:
        x[num_cols] = scaler.fit_transform(x)
        x["bin_option_type"] = oe_option_type.fit_transform(
            data["OPTION_TYPE"].astype(str).values.reshape(-1, 1)
        )
        x["bin_issue_type"] = oe_issue_type.fit_transform(
            data["issue_type"].astype(str).values.reshape(-1, 1)
        )
        x["bin_root"] = oe_root.fit_transform(
            data["ROOT"].astype(str).values.reshape(-1, 1)
        )
        print("fit_transform (train)")

    x["buy_sell"] = data["buy_sell"]
    return x


## Write to file

In [22]:
train = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_extended_60.parquet",
    engine="fastparquet",
)
val = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/val_set_extended_20.parquet",
    engine="fastparquet",
)
test = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_extended_20.parquet",
    engine="fastparquet",
)

In [23]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

name = "ise_log_standardized"
#dataset = wandb.Artifact(name=name, type="preprocessed_data")

output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/train_set_60.parquet"
)
train = transform(train)
train.to_parquet(output_path)
#dataset.add_reference(output_path)

del train
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/val_set_20.parquet"
)
val = transform(val)
val.to_parquet(output_path)
#dataset.add_reference(output_path)

del val
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/test_set_20.parquet"
)
test = transform(test)
test.to_parquet(output_path)
#dataset.add_reference(output_path)

except
try
try


In [24]:
test.describe()

,TRADE_PRICE,bid_ask_size_ratio_ex,rel_bid_size_ex,rel_ask_size_ex,depth_ex,prox_ex,prox_best,spread_ex,spread_best,bid_ask_ratio_ex,...,BEST_ASK,BEST_BID,price_all_lag,price_all_lead,price_ex_lag,price_ex_lead,TRADE_SIZE,bid_size_ex,ask_size_ex,buy_sell
count,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,...,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06
mean,-7.061342e-01,-9.998360e-01,-9.998290e-01,-9.998783e-01,-1.517021e-04,-4.496103e-01,-2.217283e-01,-3.360071e-01,-9.974931e-01,-7.037069e-01,...,-8.189535e-01,-7.144768e-01,-7.731023e-01,-8.203375e-01,-7.321574e-01,-7.479942e-01,-8.128382e-01,-4.204213e-01,-3.656733e-01,-2.805495e-02
std,2.711683e-01,1.158564e-03,1.354047e-03,8.864031e-04,3.358728e-03,6.287550e-04,7.700381e-04,6.190585e-03,7.246308e-06,1.178656e-01,...,1.602128e-01,2.718154e-01,2.112202e-01,1.696631e-01,2.708560e-01,2.639858e-01,1.995284e-01,3.001729e-01,2.637453e-01,9.996064e-01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.027665e-01,-5.057553e-01,-3.200000e-01,-1.061782e+00,-9.978192e-01,-1.000000e+00,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,-9.195079e-01,-9.999950e-01,-9.999933e-01,-9.999936e-01,-4.050061e-04,-4.499189e-01,-2.219634e-01,-3.387931e-01,-9.974964e-01,-7.333333e-01,...,-9.448681e-01,-9.294147e-01,-9.397130e-01,-9.544227e-01,-9.518099e-01,-9.648794e-01,-1.000000e+00,-5.834429e-01,-5.544227e-01,-1.000000e+00
50%,-7.777990e-01,-9.999808e-01,-9.999701e-01,-9.999773e-01,-5.000075e-06,-4.495944e-01,-2.217390e-01,-3.382184e-01,-9.974957e-01,-6.525424e-01,...,-8.607004e-01,-7.878170e-01,-8.287887e-01,-8.662185e-01,-8.065964e-01,-8.251519e-01,-8.643808e-01,-4.034553e-01,-3.925944e-01,-1.000000e+00
75%,-5.676084e-01,-9.999556e-01,-9.998640e-01,-9.998918e-01,2.650040e-04,-4.493566e-01,-2.215071e-01,-3.363218e-01,-9.974932e-01,-6.220183e-01,...,-7.362892e-01,-5.769865e-01,-6.658070e-01,-7.341432e-01,-5.959212e-01,-6.147051e-01,-6.664538e-01,-2.222069e-01,-1.915243e-01,1.000000e+00
max,1.096489e+00,-8.631786e-01,-6.186268e-02,-3.209091e-01,1.369371e-01,-4.067200e-01,1.930437e-01,1.895977e+00,-9.972290e-01,-1.000000e-01,...,2.405476e-01,1.096545e+00,5.909646e-01,2.682220e-01,1.029699e+00,1.000336e+00,9.747139e-01,6.547516e-01,6.051181e-01,1.000000e+00


In [ ]:
#run.log_artifact(dataset)
#run.finish()